In [ ]:
#Clone the Toyota Smarthome repository
!git clone https://github.com/dairui01/Toyota_Smarthome.git

Cloning into 'Toyota_Smarthome'...
remote: Enumerating objects: 470, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 470 (delta 63), reused 1 (delta 0), pack-reused 370
Receiving objects: 100% (470/470), 28.34 MiB | 17.87 MiB/s, done.
Resolving deltas: 100% (240/240), done.


In [ ]:
pip install torch==1.10.1+cu111 torchvision==0.11.2 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
pip install timm==0.4.12 scikit-learn

In [ ]:
#To mount your own google drive to put in the edited train.py and TSU_RGB_i3d_feat.zip to the file
#Need to ensure that the 2 files (train.py and TSU_RGB_i3d_feat.zip) in your google drive root folder
#Also create the directories accordingly
from google.colab import drive
drive.mount('/content/drive/')

!unzip /content/drive/My\ Drive/3104Resources/TSU_RGB_i3d_feat
!mkdir TSU_RGB_i3d_feat
!mkdir -p /content/Toyota_Smarthome/pipline/Datasets/Video
!mkdir /content/Toyota_Smarthome/pipline/Datasets/PreTrainModel
!mkdir /content/Toyota_Smarthome/pipline/Datasets/TrainedModel
!mkdir -p /content/Toyota_Smarthome/pipline/Datasets/TrainedModel/PDAN
!mv /content/RGB_i3d_16frames_64000_SSD TSU_RGB_i3d_feat
!mv /content/TSU_RGB_i3d_feat /content/Toyota_Smarthome/pipline/
!cp /content/drive/My\ Drive/3104Resources/PDAN_TSU_RGB /content/Toyota_Smarthome/pipline/Datasets/PreTrainModel
!cp -f /content/drive/My\ Drive/3104Resources/train.py /content/Toyota_Smarthome/pipline/train.py
!cp /content/drive/My\ Drive/3104Resources/sample-mp4-file.mp4 /content/Toyota_Smarthome/pipline/Datasets/Video

<h1> Function: View the video playback of the selected video inside the Datasets Folder. </h1>

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Video

#To get the current directory and set the path of the folder to the data folder
def getDatasetFolderVideoContent():
    current_dir = str(os.getcwd())
    data_folder = current_dir + "/Toyota_Smarthome/pipline/Datasets/Video"
    data_folder_content = []

    #Loop through to get all the contents inside the data folder
    with os.scandir(data_folder) as entries:
        for entry in entries:
            data_folder_content.append(entry.name)
    
    return data_folder_content

#Dropdown to display all videos in data folder
video_dropdown = widgets.Dropdown(
    options = getDatasetFolderVideoContent(),
    description = 'Videos:',
    disabled = False,
)

display(video_dropdown)


In [ ]:
#Set the selected video to the iPyWidget video function and display the video playback
selected_video = Video.from_file('/content/Toyota_Smarthome/pipline/Datasets/Video/' + video_dropdown.value)
selected_video

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\xa0\x1dTmdat\x01\x…

<h1>Model Training Sequence</h1>

In [ ]:
#model labelling function
def label_model_name(modelName, dataName):
  # removes file extensions
  extractModelFileName = modelName.split(".", 1)[0]
  extractDataFileName = dataName.split(".", 1)[0]

  # removes spaces
  strippedModelName = extractModelFileName.replace(" ", "")
  strippedDataName = extractDataFileName.replace(" ", "")

  # combines both model and data name , e.g. tsu_data1
  newLabel = strippedModelName + "_" + strippedDataName

  return newLabel

In [ ]:
from ipywidgets import Layout, interact, interact_manual, fixed
import IPython.display as display
from IPython.display import clear_output

def on_button_clicked(b):
    batch_size_value = batch_size.value
    epoch_value = epoch.value
    argv_dict["batch_size"] = batch_size_value
    argv_dict["epoch"] = epoch_value
    print("Values Set: ")
    print("Batch Size: ", batch_size_value)
    print("Epoch: ", epoch_value)
    print("Pre Train Model Selected: ", pretrainedModel.value)

def on_clear_clicked(b):
    clear_output(wait=False)

def sidebyside(list1):
  side2side = widgets.HBox(list1)
  display.display(side2side)
  return list1

def batchButtonClick(side2side):
  button.on_click(on_button_clicked)
  clear.on_click(on_clear_clicked)

def getDatasetFolderPreTrainModel():
    current_dir = str(os.getcwd())
    data_folder = current_dir + "/Toyota_Smarthome/pipline/Datasets/PreTrainModel"
    data_folder_content = []

    #Loop through to get all the contents inside the data folder
    with os.scandir(data_folder) as entries:
        for entry in entries:
            data_folder_content.append(entry.name)
    
    return data_folder_content

In [ ]:
argv_dict = {}
flag = False

epoch = widgets.IntSlider(
    value=20,
    min=5,
    max=100,
    step=5,
    description='Epoch:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

batch_size = widgets.Dropdown(
    value = '1',
    placeholder='Choose Batch Size',
    options=['1','2', '4', '8', '16', '32', '64'],
    description='batch_size:',
    ensure_option=True,
    disabled=False
)

pretrainedModel = widgets.Dropdown(
    options = getDatasetFolderPreTrainModel(),
    description = 'Pretrain Models:',
    disabled = False,
)


button = widgets.Button(description="Add",icon='check', command=on_button_clicked)
clear = widgets.Button(description="Clear",icon='check', command=on_clear_clicked)


In [ ]:
#Change directory to pipline directory to run the training sequence and change back to root directory
os.getcwd()
os.chdir('/content/Toyota_Smarthome/pipline')
pretrained_model = "Datasets/PreTrainModel/" + pretrainedModel.value
user_input_epoch = argv_dict["epoch"]
user_input_batch_size = argv_dict["batch_size"]

!python train.py -dataset TSU -mode rgb -split_setting CS -model PDAN -train True -num_channel 512 -lr 0.0002 -kernelsize 3 -APtype map -epoch $user_input_epoch -batch_size $user_input_batch_size -comp_info TSU_CS_RGB_PDAN -load_model ./{pretrained_model} 
os.chdir('/content')